In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [0]:
def area(box):
    return (box[2] - box[0]) * (box[3] - box[1])


def intersection_over_union(boxes):
    assert(len(boxes) == 8)
    boxA = boxes[:4].values
    boxB = boxes[4:].values
    
    boxAArea = area(boxA)
    boxBArea = area(boxB)
    
    if (boxAArea == 0 or boxBArea == 0):
        return 0
        
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [0]:
from google.colab import drive
drive.mount('drive')

In [13]:
votes = pd.read_csv("drive/My Drive/sna9/train3_data.csv")
votes.head()

,userId,itemId,Xmin,Ymin,Xmax,Ymax,Xmin_true,Ymin_true,Xmax_true,Ymax_true,acc,se_Xmin,se_Ymin,se_Xmax,se_Ymax,Y_xmin,Y_ymin,Y_xmax,Y_ymax,L,H,P,S,L/H,D,iou,best
0,1164,6132,62,656,276,803,47,650,213,812,0.988892,225,36,3969,81,1,1,0,1,214.0,147.0,722.0,31458.0,1.455782,259.624729,0.613974,1
1,1399,6132,184,662,235,740,47,650,213,812,0.936663,18769,144,484,5184,0,0,1,0,51.0,78.0,258.0,3978.0,0.653846,93.193347,0.079069,0
2,1694,6132,63,648,272,830,47,650,213,812,0.989526,256,4,3481,324,1,1,0,1,209.0,182.0,782.0,38038.0,1.148352,277.137150,0.598080,1
3,916,6132,111,637,253,831,47,650,213,812,0.983958,4096,169,1600,361,0,0,1,1,142.0,194.0,672.0,27548.0,0.731959,240.416306,0.435805,0
4,1621,6132,60,651,242,819,47,650,213,812,0.997269,169,1,841,49,1,1,1,1,182.0,168.0,700.0,30576.0,1.083333,247.685284,0.750206,1


In [0]:
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier

Есть два пути: 


1.   обучить на всем наборе и потом выбрать maxbox'ы
2.   сначала  maxbox'ы, потом обучить

# 1 путь

In [28]:
X = votes[['Xmin','Ymin','Xmax','Ymax', 'L','H','P','S','L/H','D']]
y = votes.best
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
reg = AdaBoostClassifier(base_estimator=GradientBoostingClassifier(n_estimators=200)).fit(X_train, y_train)
reg.score(X_test, y_test) # 0.6860902255639098 0.7048872180451128

0.7058270676691729

ок. Теперь проверка

In [29]:
data = votes[['userId','itemId','Xmin','Ymin','Xmax','Ymax']]
data['pred'] = reg.predict(X)
data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,userId,itemId,Xmin,Ymin,Xmax,Ymax,pred
0,1164,6132,62,656,276,803,1
1,1399,6132,184,662,235,740,0
2,1694,6132,63,648,272,830,1
3,916,6132,111,637,253,831,0
4,1621,6132,60,651,242,819,1


In [30]:
Xminmin = data.groupby("itemId")['Xmin'].min().reset_index().set_index('itemId')
Yminmin = data.groupby("itemId")['Ymin'].min().reset_index().set_index('itemId')
Xmaxmax = data.groupby("itemId")['Xmax'].max().reset_index().set_index('itemId')
Ymaxmax = data.groupby("itemId")['Ymax'].max().reset_index().set_index('itemId')
    
maxbox = pd.concat([Xminmin,Yminmin,Xmaxmax,Ymaxmax], axis=1)
trubox = votes.groupby("itemId")[['Xmin_true','Ymin_true','Xmax_true','Ymax_true']].min().reset_index().set_index('itemId')
data = pd.concat([maxbox, trubox], axis=1)
    
data["iou"] = data.apply(intersection_over_union, axis=1)
print(data["iou"].mean()) #0.5822325332074827

0.5822325332074827


...хм. Вообще говоря - не лучше, чем было. Интересно - а если взять средние?

In [21]:
data = data.groupby("itemId")[['Xmin','Ymin','Xmax','Ymax']].mean().reset_index().set_index('itemId')
data = pd.concat([data, trubox], axis=1)
data["iou"] = data.apply(intersection_over_union, axis=1)
print(data["iou"].mean())       0.5068380088328308 NO!!!!                    

0.5068380088328308


НЕТ. Средние нах, а вот поэкспериментировать с количеством 1 стоит 

# 2 путь